In [1]:
from datasets import load_dataset, load_metric
import datasets
from IPython.display import display, HTML
import numpy as np
import pandas as pd
import random
import torch
from torch.optim import Adam

import transformers
from transformers import AdamW,AutoTokenizer, AutoModelForSequenceClassification,get_linear_schedule_with_warmup, TrainingArguments, Trainer
from torch.optim import Adadelta, Adam, SGD, RMSprop, AdamW
from transformers import AdamW
#from transformers.trainer_utils import A

2023-01-02 14:27:16.532532: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-02 14:27:17.277485: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-02 14:27:17.277541: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-02 14:27:17.277548: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [4]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

In [5]:
task = "mnli"
model_checkpoint = "microsoft/mpnet-base"#"symanto/mpnet-base-snli-mnli"


In [6]:
actual_task = "mnli" if task == "mnli-mm" else task
dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', actual_task)

Using the latest cached version of the module from /home/nico/.cache/huggingface/modules/datasets_modules/datasets/glue/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad (last modified on Mon Dec 12 10:10:25 2022) since it couldn't be found locally at glue., or remotely on the Hugging Face Hub.
Found cached dataset glue (/home/nico/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/5 [00:00<?, ?it/s]

/tmp/ipykernel_125784/1389288479.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', actual_task)
Using the latest cached version of the module from /home/nico/.cache/huggingface/modules/datasets_modules/metrics/glue/91f3cfc5498873918ecf119dbf806fb10815786c84f41b85a5d3c47c1519b343 (last modified on Mon Dec 12 10:42:07 2022) since it couldn't be found locally at glue, or remotely on the Hugging Face Hub.


In [7]:
#%pip install scikit-learn

In [8]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
show_random_elements(dataset["train"])

,premise,hypothesis,label,idx
0,"He thought things over in his usual slow and steady way, deciding that the mention of ""Mr. Brown"" was not a request for an individual, but in all probability a password used by the gang.",After careful thought he determined it was most likely a password used by the gang.,entailment,168658
1,"Oh, and let's cut out all this huffing and puffing about Saddam Hussein.",People are pleased with Hussein.,contradiction,17803
2,"In 1992 the Federal Communications Commission (FCC) adopted rules to implement the requirements of the Hearing Aid Compatibility Act of 1988, 47 U.S.C.",The FCC adopted rules about implementing the requirements of the act.,entailment,228080
3,"After four students complained to the administration about her behavior in the classroom last spring, Sandra Flitterman-Lewis was asked to sign a release form requiring her to be medically examined before teaching again this fall.",Sandra Flitterman-Lewis had to have a medical examination before teaching again this fall because students reported her behavior in class.,entailment,79609
4,"More than just Japan's second city, Osaka is also the perfect base from which to explore nearby Nara and Kyoto by train.","If one wants to explore Nara and Kyoto, Osaka is the best place to do it from.",neutral,256373
5,"15 Since NIPA focuses on the current production of goods and services and on the income arising from that production, NIPA income and saving do not reflect changes in the value of existing tangible and financial assets, such as land, stocks, or bonds.",Stocks and bonds are not tangible financial assets.,contradiction,214873
6,"There was general agreement among the participants that the root causes of the accountability breakdowns are systemic in nature, complex, and will require leadership and alterations to the current models in each of the four interrelated areas to transition to an overall system that is more focused on protecting the public interest and, in that regard, accountability.",Accountability issues are not systemic.,contradiction,294920
7,Their tough-minded pragmatism survives today not only in Tokyo but also in the dynamic industrial zone that has burgeoned around it in such towns as Kawasaki and Yokohama.,Yokohama is part of the industrial zone around Tokyo.,entailment,12751
8,"A short walk from Uji-Yamada Station is Geku, the Outer Shrine.",Uji-Yamada is an airport station.,contradiction,118375
9,Jerusalem is an ever-changing combination of the earthly and the spiritual.,Jerusalem is an ever shifting combination of the spiritual and the earthly.,entailment,174601


In [9]:
#print(metric)

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
print(metric.compute(predictions=fake_preds, references=fake_labels,))


{'accuracy': 0.53125}


In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [11]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.", truncation=True)

{'input_ids': [0, 7596, 1014, 2027, 2032, 6255, 1003, 2, 2, 2002, 2027, 6255, 3636, 2011, 2013, 1016, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [12]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

In [13]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence 1: Conceptually cream skimming has two basic dimensions - product and geography.
Sentence 2: Product and geography are what make cream skimming work. 


In [14]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

In [15]:
preprocess_function(dataset['train'][:5])

{'input_ids': [[0, 17162, 2139, 6953, 8305, 25061, 2042, 2052, 3941, 9650, 1015, 4035, 2002, 10509, 1016, 2, 2, 4035, 2002, 10509, 2028, 2058, 2195, 6953, 8305, 25061, 2151, 1016, 2], [0, 2021, 2117, 2080, 2000, 2165, 2002, 1049, 3988, 2016, 2016, 2119, 2508, 7914, 2021, 4562, 2072, 2004, 2000, 2283, 2508, 2069, 2069, 2031, 5634, 2004, 9135, 2000, 2000, 6691, 2140, 2000, 13984, 5634, 2004, 2659, 2004, 9135, 1041, 3128, 2017, 6424, 1041, 2063, 1041, 3317, 1041, 3128, 3636, 2043, 2004, 5676, 2036, 2002, 1041, 2313, 1041, 3128, 3636, 2043, 2004, 5676, 2036, 2, 2, 2021, 4562, 2000, 2481, 2004, 2000, 2210, 2508, 2069, 2000, 2115, 9135, 1016, 2], [0, 2032, 2001, 2260, 2197, 2101, 4291, 2045, 2119, 8132, 3375, 2139, 1016, 2, 2, 1041, 2270, 2001, 2030, 2140, 2101, 15393, 2119, 4453, 2011, 14273, 11722, 1016, 2], [0, 2133, 2083, 2021, 2117, 1033, 2039, 2027, 2007, 2041, 2596, 2157, 1016, 2, 2, 2027, 2596, 7464, 2004, 2072, 1016, 2], [0, 3402, 1049, 2429, 2021, 2058, 2299, 2069, 2021, 2179, 3980

Preprocessing

In [16]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Loading cached processed dataset at /home/nico/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-006a0f5c7c2c7c19.arrow
Loading cached processed dataset at /home/nico/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-6a827a948b880c8b.arrow
Loading cached processed dataset at /home/nico/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-fd6a11c8c4e8700f.arrow
Loading cached processed dataset at /home/nico/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-1b1b3b19ab22acaf.arrow
Loading cached processed dataset at /home/nico/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-b38687898c35c284.arrow


In [17]:

num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at microsoft/mpnet-base were not used when initializing MPNetForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at microsoft/mpnet-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classi

In [18]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]
batch_size = 4
optimizer_name = "AdamW" #AdamW, SGD, Adadelta
args = TrainingArguments(
    f"{model_name}-finetuned-{optimizer_name}-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2.7e-3 , #3e-5 AdamW
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=0.20,
    weight_decay=0.5,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

In [19]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [20]:
#estimated_training_size =args.size_of_dataset if args.size_of_dataset is not None else len(encoded_dataset["train"])
estimated_training_size = len(encoded_dataset["train"])
num_epoch = args.num_train_epochs*100 if args.num_train_epochs is not None else 1

print(num_epoch, estimated_training_size,args.num_train_epochs)
total_step = estimated_training_size * num_epoch // args.per_device_train_batch_size
if total_step <= 0:
        t_total = estimated_training_size * num_epoch // args.actual_train_batch_size
else:
    t_total = total_step

if args.warmup_steps <= 0:  # set the warmup steps to 0.1 * total step if the given warmup step is -1.
    args.warmup_steps = int(t_total * 0.1)
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": args.weight_decay,
    },
    {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
]

optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
#optimizer = SGD(optimizer_grouped_parameters, lr=args.learning_rate, momentum=0.0, weight_decay=args.weight_decay) #pas reussi lr=3e-6 3e-5 3e-4 3e-3 3e-2
#optimizer=Adadelta(optimizer_grouped_parameters, lr=args.learning_rate, weight_decay=args.weight_decay,) #lr=3e-6 
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
)

20.0 392702 0.2


/home/nico/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [21]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    #optimizers=(optimizer, scheduler),
    #optimizers=(transformers.AdamW, transformers.get_scheduler("linear", num_warmup_steps=0, num_training_steps=args.num_train_epochs*len(encoded_dataset["train"])/args.per_device_train_batch_size)),
    #token=token
)

ConnectionError: HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /api/whoami-v2 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f9940230130>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `MPNetForSequenceClassification.forward` and have been ignored: premise, hypothesis, idx. If premise, hypothesis, idx are not expected by `MPNetForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 392702
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 19636
  Number of trainable parameters = 109488771


  0%|          | 0/19636 [00:00<?, ?it/s]

You're using a MPNetTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 1.0987, 'learning_rate': 6.875442447453794e-06, 'epoch': 0.01}
{'loss': 0.994, 'learning_rate': 1.3750884894907589e-05, 'epoch': 0.01}
{'loss': 0.775, 'learning_rate': 2.0626327342361386e-05, 'epoch': 0.02}
{'loss': 0.7372, 'learning_rate': 2.7501769789815177e-05, 'epoch': 0.02}
{'loss': 0.7932, 'learning_rate': 3.437721223726897e-05, 'epoch': 0.03}
{'loss': 0.7943, 'learning_rate': 4.125265468472277e-05, 'epoch': 0.03}
{'loss': 0.7841, 'learning_rate': 4.812809713217657e-05, 'epoch': 0.04}
{'loss': 0.836, 'learning_rate': 5.5003539579630354e-05, 'epoch': 0.04}
{'loss': 0.8565, 'learning_rate': 6.187898202708416e-05, 'epoch': 0.05}
{'loss': 0.9539, 'learning_rate': 6.875442447453794e-05, 'epoch': 0.05}
{'loss': 1.0525, 'learning_rate': 7.562986692199174e-05, 'epoch': 0.06}
{'loss': 1.1059, 'learning_rate': 8.250530936944554e-05, 'epoch': 0.06}
{'loss': 1.1016, 'learning_rate': 8.938075181689933e-05, 'epoch': 0.07}
{'loss': 1.1037, 'learning_rate': 9.625619426435313e-05, 'epoch

The following columns in the evaluation set don't have a corresponding argument in `MPNetForSequenceClassification.forward` and have been ignored: premise, hypothesis, idx. If premise, hypothesis, idx are not expected by `MPNetForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9815
  Batch size = 4


...

Saving model checkpoint to mpnet-base-finetuned-AdamW-mnli/checkpoint-19636
Configuration saved in mpnet-base-finetuned-AdamW-mnli/checkpoint-19636/config.json


{'eval_loss': 1.0983335971832275, 'eval_accuracy': 0.3273560876209883, 'eval_runtime': 43.3066, 'eval_samples_per_second': 226.64, 'eval_steps_per_second': 56.666, 'epoch': 0.2}


Model weights saved in mpnet-base-finetuned-AdamW-mnli/checkpoint-19636/pytorch_model.bin
tokenizer config file saved in mpnet-base-finetuned-AdamW-mnli/checkpoint-19636/tokenizer_config.json
Special tokens file saved in mpnet-base-finetuned-AdamW-mnli/checkpoint-19636/special_tokens_map.json
tokenizer config file saved in mpnet-base-finetuned-AdamW-mnli/tokenizer_config.json
Special tokens file saved in mpnet-base-finetuned-AdamW-mnli/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from mpnet-base-finetuned-AdamW-mnli/checkpoint-19636 (score: 0.3273560876209883).


{'train_runtime': 2003.703, 'train_samples_per_second': 39.198, 'train_steps_per_second': 9.8, 'train_loss': 1.0381313843842703, 'epoch': 0.2}


TrainOutput(global_step=19636, training_loss=1.0381313843842703, metrics={'train_runtime': 2003.703, 'train_samples_per_second': 39.198, 'train_steps_per_second': 9.8, 'train_loss': 1.0381313843842703, 'epoch': 0.2})

In [ ]:
print(metric.compute(predictions=fake_preds, references=fake_labels,))
#Trainer.push_to_hub(trainer, model_id=f"{model_name}-finetuned-{task}")

{'accuracy': 0.484375}


In [ ]:
#fitetuningpretrained pytorch from scratch
#del model
#del trainer
#del args
#del batch
#torch.cuda.empty_cache()

: 